In [1]:
import numpy as np
import pandas as pd
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
ball = pd.read_csv('Half Ball Prediction Data.csv')
convex = pd.read_csv('Convex Prediction Data.csv')
freeform = pd.read_csv('Freeform 2 Prediction Data.csv')

In [3]:
test_data = pd.read_csv("Whole Test Grid.csv")

In [4]:
test_total = []
for i in range(1,51):
    nameX = "Scan" + str(i) + "X"
    nameY = "Scan" + str(i) + "Y"
    nameZ = "Scan" + str(i) + "Z"
    nameCurv = "Scan" + str(i) + "curvature"
    nameAngle = "Scan" + str(i) + "angle"
    scan_i = test_data[[nameX, nameY, nameZ,nameCurv,nameAngle]]
    scan_i.columns = ['X', 'Y', 'Z','curvature','angle']
    test_total.append(scan_i)

test_d = test_total[2]

In [5]:
# parameter space:
## curvature: [0,0.4]
## angle: [-1, 1]

In [6]:
curvature_step = np.linspace(start=0,stop=0.4,num=30)
angle_step = np.linspace(start=-1,stop=1,num=30)

In [7]:
ball_para = ball[['Mean Curvature','Mean Angle']]
convex_para = convex[['Mean Curvature','Mean Angle']]
freeform_para = freeform[['Mean Curvature','Mean Angle']]

In [8]:
def cal_dist(data, curvature_step, angle_step):
    num_data = len(data)
    dist = []
    for i in range(len(curvature_step)-1):
        for j in range(len(angle_step)-1):
            flag = (data['Mean Curvature'] > curvature_step[i]) & (data['Mean Curvature'] <= curvature_step[i+1]) & (data['Mean Angle'] > angle_step[i]) & (data['Mean Angle'] < angle_step[i+1])
            freq = len(data[flag])
            dist.append(freq / num_data)
    dist = np.array(dist)
    return(dist)

In [9]:
ball_dist = cal_dist(ball_para,curvature_step,angle_step)
convex_dist = cal_dist(convex_para,curvature_step,angle_step)
freeform_dist = cal_dist(freeform_para,curvature_step,angle_step)

In [10]:
def cal_dist_2(data, curvature_step, angle_step):
    num_data = len(data)
    dist = []
    for i in range(len(curvature_step)-1):
        for j in range(len(angle_step)-1):
            flag = (data['curvature'] > curvature_step[i]) & (data['curvature'] <= curvature_step[i+1]) & (data['angle'] > angle_step[i]) & (data['angle'] < angle_step[i+1])
            freq = len(data[flag])
            dist.append(freq / num_data)
    dist = np.array(dist)
    return(dist)

In [11]:
test_dist = cal_dist_2(test_d,curvature_step,angle_step)

In [12]:
def kullback_leibler_divergence(p, q):
    sum_val = 0
    for i in range(len(q)):
        if q[i] == 0:
            continue
        else:
            if p[i] == 0:
                sum_val +=-1000*q[i]
            else:
                sum_val += p[i] * np.log(p[i]/q[i])
                
    return sum_val

In [17]:
# ball vs others
print(kullback_leibler_divergence(convex_dist,ball_dist))
print(kullback_leibler_divergence(test_dist,ball_dist))
print(kullback_leibler_divergence(freeform_dist,ball_dist))

-0.17471146648360522
-1083.9833144196975
-667.2340358966509


In [18]:
# convex vs others
print(kullback_leibler_divergence(ball_dist,convex_dist))
print(kullback_leibler_divergence(test_dist,convex_dist))
print(kullback_leibler_divergence(freeform_dist,convex_dist))

-422.07269342543646
-17.49449799103283
-13.269790362738586


In [19]:
# freeform vs others
print(kullback_leibler_divergence(ball_dist,test_dist))
print(kullback_leibler_divergence(convex_dist,test_dist))
print(kullback_leibler_divergence(freeform_dist,test_dist))

-466.30285266910204
-1.8423535646695794
-24.288891481300674


In [20]:
# freeform 2 vs others
print(kullback_leibler_divergence(ball_dist,freeform_dist))
print(kullback_leibler_divergence(convex_dist,freeform_dist))
print(kullback_leibler_divergence(test_dist,freeform_dist))

-527.7145643581258
0.08856113724197422
-3.7542934148826315


In [13]:
kullback_leibler_divergence(ball_dist,test_dist)

-466.30285266910204

In [14]:
kullback_leibler_divergence(convex_dist,test_dist)

-1.8423535646695794

In [15]:
kullback_leibler_divergence(freeform_dist,test_dist)

-24.288891481300674

In [16]:
kld = pd.DataFrame()

In [ ]:

for num in np.linspace(5,40,num=20):
    curvature_step = np.linspace(start=0,stop=0.4,num=num)
    angle_step = np.linspace(start=-1,stop=1,num=num)
    